In [1]:
import json
import requests
import pandas as pd
import os
import sys
from datetime import datetime
from pandas.io.json import json_normalize
from math import ceil

In [2]:
#diangose intraday API issues using UAL

In [40]:
#setup
run_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
print('Starting run, timestamp is: ' + run_timestamp)

#create the data directory
dirname = os.getcwd() + '/data/pull_' + run_timestamp
os.mkdir(dirname)
print('Data directory created:' + dirname)

output_df = pd.DataFrame()

tickers = ['MMM','AOS','ABT','ABBV','ABMD','ACN','ATVI','ADBE','AAP','AMD','AES','AFL','A','APD','AKAM','ALK','ALB','ARE','ALXN','ALGN','ALLE','LNT','ALL','GOOGL','GOOG','MO','AMZN','AMCR','AEE','AAL','AEP','AXP','AIG','AMT','AWK','AMP','ABC','AME','AMGN','APH','ADI','ANSS','ANTM','AON','APA','AIV','AAPL','AMAT','APTV','ADM','ANET','AJG','AIZ','T','ATO','ADSK','ADP','AZO','AVB','AVY','BKR','BLL','BAC','BAX','BDX','BBY','BIO','BIIB','BLK','BA','BKNG','BWA','BXP','BSX','BMY','AVGO','BR','CHRW','COG','CDNS','CPB','COF','CAH','KMX','CCL','CARR','CAT','CBOE','CBRE','CDW','CE','CNC','CNP','CTL','CERN','CF','SCHW','CHTR','CVX','CMG','CB','CHD','CI','CINF','CTAS','CSCO','C','CFG','CTXS','CME','CMS','KO','CTSH','CL','CMCSA','CMA','CAG','CXO','COP','ED','STZ','CPRT','GLW','CTVA','COST','COTY','CCI','CSX','CMI','CVS','DHI','DHR','DRI','DVA','DE','DAL','DVN','DXCM','FANG','DLR','DFS','DISCA','DISCK','DISH','DG','DLTR','D','DPZ','DOV','DOW','DTE','DUK','DRE','DD','DXC','ETFC','EMN','ETN','EBAY','ECL','EIX','EW','EA','EMR','ETR','EOG','EFX','EQIX','EQR','ESS','EL','RE','EVRG','ES','EXC','EXPE','EXPD','EXR','FFIV','FB','FAST','FRT','FDX','FIS','FITB','FRC','FE','FISV','FLT','FLIR','FLS','FMC','F','FTNT','FTV','FBHS','FOXA','FOX','BEN','FCX','GPS','GRMN','IT','GD','GE','GIS','GM','GPC','GILD','GPN','GL','GS','GWW','HRB','HAL','HBI','HIG','HAS','HCA','PEAK','HSIC','HES','HPE','HLT','HFC','HOLX','HD','HON','HRL','HST','HPQ','HUM','HBAN','HII','IEX','IDXX','INFO','ITW','ILMN','INCY','IR','INTC','ICE','IBM','IFF','IP','IPG','INTU','ISRG','IVZ','IPGP','IQV','IRM','JBHT','JKHY','J','SJM','JNJ','JCI','JPM','JNPR','KSU','K','KEY','KEYS','KMB','KIM','KMI','KLAC','KSS','KHC','KR','LB','LHX','LH','LRCX','LW','LVS','LEG','LDOS','LEN','LLY','LNC','LIN','LYV','LKQ','LMT','L','LOW','LYB','MTB','MRO','MPC','MKTX','MAR','MMC','MLM','MAS','MA','MXIM','MKC','MCD','MCK','MDT','MRK','MET','MTD','MGM','MCHP','MU','MSFT','MAA','MHK','TAP','MDLZ','MNST','MCO','MS','MSI','MSCI','MYL','NDAQ','NOV','NTAP','NFLX','NWL','NEM','NWSA','NWS','NEE','NLSN','NKE','NI','NBL','NSC','NTRS','NOC','NLOK','NCLH','NRG','NUE','NVDA','NVR','ORLY','OXY','ODFL','OMC','OKE','ORCL','PCAR','PKG','PH','PAYX','PAYC','PYPL','PNR','PBCT','PEP','PKI','PRGO','PFE','PM','PSX','PNW','PXD','PNC','PPG','PPL','PFG','PG','PGR','PLD','PRU','PEG','PSA','PHM','PVH','QRVO','QCOM','PWR','DGX','RL','RJF','RTX','O','REG','REGN','RF','RSG','RMD','RHI','ROK','ROL','ROP','ROST','RCL','SPGI','CRM','SBAC','SLB','STX','SEE','SRE','NOW','SHW','SPG','SWKS','SLG','SNA','SO','LUV','SWK','SBUX','STT','STE','SYK','SIVB','SYF','SNPS','SYY','TMUS','TROW','TTWO','TPR','TGT','TEL','FTI','TDY','TFX','TXN','TXT','BK','CLX','COO','HSY','MOS','TRV','DIS','TMO','TIF','TJX','TSCO','TDG','TFC','TWTR','TYL','TSN','USB','UDR','ULTA','UAA','UA','UNP','UAL','UNH','UPS','URI','UHS','UNM','VLO','VAR','VRSN','VRSK','VRTX','VFC','VIAC','V','VNO','VMC']

def downloadOneStock(stock_num, ticker_symbol):
    print('------\nStarting stock #' + str(stock_num) + ' with ticker ' + ticker_symbol)

    return_df = pd.DataFrame()
    
    stock_params = {
          'access_key': 'removed',
          'date_from': '2016-08-01',
          'date_to': '2021-03-05',
          'limit': 1000,
          'offset': 0
    }
    stock_params['symbols'] = ticker_symbol
    
#     print("Sending Intraday request...")
#     print(stock_params)
    
    api_result = requests.get('http://api.marketstack.com/v1/intraday', stock_params)
    if api_result.status_code != 200:
        print("API error %d, ending program" % (api_result.status_code))
        print(api_result)
        sys.exit()
    json_result = api_result.json()

    json_normalized = pd.json_normalize(json_result)
    total = json_normalized['pagination.total'][0]
    limit = json_normalized['pagination.limit'][0]
    offset = json_normalized['pagination.offset'][0]
    total_calls = ceil(1.0* total / limit)
    
    print('Available: %d | Limit: %d | Calls needed: %d' % (total, limit, total_calls))

    csv_name = dirname + "/apidata_s_" + ticker_symbol + "_" + str(0) + "_" + run_timestamp + ".csv"
    pd.json_normalize(json_result['data']).to_csv(csv_name)
#     print("File written: " + csv_name)
    
    #add to results DF
    return_df = pd.json_normalize(json_result['data'])
    
    for n in range(1,total_calls):
        #form request
        request_offset = n*limit
        request_params = stock_params
        request_params['offset'] = request_offset
        
        #send request
#         print("Sending API request...")
#         print(request_params)
        this_request = requests.get('http://api.marketstack.com/v1/intraday', request_params)
        if this_request.status_code != 200:
            print("API error, breaking.")
            break

        #save to disk
        tr_json = this_request.json()
        this_filename = dirname + "/apidata_s_" + ticker_symbol + "_" + str(n) + "_" + run_timestamp + ".csv"
        pd.json_normalize(tr_json['data']).to_csv(this_filename)
#         print("File written: " + this_filename)
        
        #append to results DF
        return_df = return_df.append(pd.json_normalize(tr_json['data']), ignore_index=True)
    
    return return_df

for i in range(len(tickers)):
    output_df = output_df.append(downloadOneStock(i,tickers[i]), ignore_index=True)

final_output_filename = dirname + "/full_intraday_data_" + run_timestamp + ".csv"
output_df.to_csv(final_output_filename)
print("File written: " + final_output_filename)



Starting run, timestamp is: 20210306_162459
Data directory created:/Users/mike/Desktop/ml_final/downloader/data/pull_20210306_162459
------
Starting stock #0 with ticker MMM
Available: 1026 | Limit: 1000 | Calls needed: 2
------
Starting stock #1 with ticker AOS
Available: 943 | Limit: 1000 | Calls needed: 1
------
Starting stock #2 with ticker ABT
Available: 4769 | Limit: 1000 | Calls needed: 5
------
Starting stock #3 with ticker ABBV
Available: 4821 | Limit: 1000 | Calls needed: 5
------
Starting stock #4 with ticker ABMD
Available: 4465 | Limit: 1000 | Calls needed: 5
------
Starting stock #5 with ticker ACN
Available: 4734 | Limit: 1000 | Calls needed: 5
------
Starting stock #6 with ticker ATVI
Available: 1149 | Limit: 1000 | Calls needed: 2
------
Starting stock #7 with ticker ADBE
Available: 4783 | Limit: 1000 | Calls needed: 5
------
Starting stock #8 with ticker AAP
Available: 4679 | Limit: 1000 | Calls needed: 5
------
Starting stock #9 with ticker AMD
Available: 1187 | Limi

------
Starting stock #90 with ticker CE
Available: 4573 | Limit: 1000 | Calls needed: 5
------
Starting stock #91 with ticker CNC
Available: 1082 | Limit: 1000 | Calls needed: 2
------
Starting stock #92 with ticker CNP
Available: 1087 | Limit: 1000 | Calls needed: 2
------
Starting stock #93 with ticker CTL
Available: 410 | Limit: 1000 | Calls needed: 1
------
Starting stock #94 with ticker CERN
Available: 4706 | Limit: 1000 | Calls needed: 5
------
Starting stock #95 with ticker CF
Available: 4684 | Limit: 1000 | Calls needed: 5
------
Starting stock #96 with ticker SCHW
Available: 4814 | Limit: 1000 | Calls needed: 5
------
Starting stock #97 with ticker CHTR
Available: 4685 | Limit: 1000 | Calls needed: 5
------
Starting stock #98 with ticker CVX
Available: 1156 | Limit: 1000 | Calls needed: 2
------
Starting stock #99 with ticker CMG
Available: 858 | Limit: 1000 | Calls needed: 1
------
Starting stock #100 with ticker CB
Available: 4672 | Limit: 1000 | Calls needed: 5
------
Star

Available: 1085 | Limit: 1000 | Calls needed: 2
------
Starting stock #181 with ticker FRT
Available: 871 | Limit: 1000 | Calls needed: 1
------
Starting stock #182 with ticker FDX
Available: 1089 | Limit: 1000 | Calls needed: 2
------
Starting stock #183 with ticker FIS
Available: 1145 | Limit: 1000 | Calls needed: 2
------
Starting stock #184 with ticker FITB
Available: 1111 | Limit: 1000 | Calls needed: 2
------
Starting stock #185 with ticker FRC
Available: 942 | Limit: 1000 | Calls needed: 1
------
Starting stock #186 with ticker FE
Available: 1089 | Limit: 1000 | Calls needed: 2
------
Starting stock #187 with ticker FISV
Available: 1128 | Limit: 1000 | Calls needed: 2
------
Starting stock #188 with ticker FLT
Available: 947 | Limit: 1000 | Calls needed: 1
------
Starting stock #189 with ticker FLIR
Available: 862 | Limit: 1000 | Calls needed: 1
------
Starting stock #190 with ticker FLS
Available: 881 | Limit: 1000 | Calls needed: 1
------
Starting stock #191 with ticker FMC
Av

------
Starting stock #271 with ticker KR
Available: 1159 | Limit: 1000 | Calls needed: 2
------
Starting stock #272 with ticker LB
Available: 1079 | Limit: 1000 | Calls needed: 2
------
Starting stock #273 with ticker LHX
Available: 1031 | Limit: 1000 | Calls needed: 2
------
Starting stock #274 with ticker LH
Available: 945 | Limit: 1000 | Calls needed: 1
------
Starting stock #275 with ticker LRCX
Available: 1073 | Limit: 1000 | Calls needed: 2
------
Starting stock #276 with ticker LW
Available: 909 | Limit: 1000 | Calls needed: 1
------
Starting stock #277 with ticker LVS
Available: 1120 | Limit: 1000 | Calls needed: 2
------
Starting stock #278 with ticker LEG
Available: 853 | Limit: 1000 | Calls needed: 1
------
Starting stock #279 with ticker LDOS
Available: 938 | Limit: 1000 | Calls needed: 1
------
Starting stock #280 with ticker LEN
Available: 1065 | Limit: 1000 | Calls needed: 2
------
Starting stock #281 with ticker LLY
Available: 1076 | Limit: 1000 | Calls needed: 2
-----

Available: 1131 | Limit: 1000 | Calls needed: 2
------
Starting stock #362 with ticker PSX
Available: 1092 | Limit: 1000 | Calls needed: 2
------
Starting stock #363 with ticker PNW
Available: 852 | Limit: 1000 | Calls needed: 1
------
Starting stock #364 with ticker PXD
Available: 1045 | Limit: 1000 | Calls needed: 2
------
Starting stock #365 with ticker PNC
Available: 1085 | Limit: 1000 | Calls needed: 2
------
Starting stock #366 with ticker PPG
Available: 1059 | Limit: 1000 | Calls needed: 2
------
Starting stock #367 with ticker PPL
Available: 1039 | Limit: 1000 | Calls needed: 2
------
Starting stock #368 with ticker PFG
Available: 955 | Limit: 1000 | Calls needed: 1
------
Starting stock #369 with ticker PG
Available: 1146 | Limit: 1000 | Calls needed: 2
------
Starting stock #370 with ticker PGR
Available: 1091 | Limit: 1000 | Calls needed: 2
------
Starting stock #371 with ticker PLD
Available: 1109 | Limit: 1000 | Calls needed: 2
------
Starting stock #372 with ticker PRU
Av

------
Starting stock #452 with ticker UNP
Available: 1078 | Limit: 1000 | Calls needed: 2
------
Starting stock #453 with ticker UAL
Available: 1110 | Limit: 1000 | Calls needed: 2
------
Starting stock #454 with ticker UNH
Available: 1097 | Limit: 1000 | Calls needed: 2
------
Starting stock #455 with ticker UPS
Available: 1089 | Limit: 1000 | Calls needed: 2
------
Starting stock #456 with ticker URI
Available: 973 | Limit: 1000 | Calls needed: 1
------
Starting stock #457 with ticker UHS
Available: 845 | Limit: 1000 | Calls needed: 1
------
Starting stock #458 with ticker UNM
Available: 957 | Limit: 1000 | Calls needed: 1
------
Starting stock #459 with ticker VLO
Available: 4735 | Limit: 1000 | Calls needed: 5
------
Starting stock #460 with ticker VAR
Available: 4436 | Limit: 1000 | Calls needed: 5
------
Starting stock #461 with ticker VRSN
Available: 4569 | Limit: 1000 | Calls needed: 5
------
Starting stock #462 with ticker VRSK
Available: 4593 | Limit: 1000 | Calls needed: 5
